In [1]:
def compute_terrain_corrected_irradiance(
    aod550, tcwv, elevation,
    solar_zenith_deg, slope_deg, aspect_deg, solar_azimuth_deg,
    svf=1.0, direct_ratio=0.8
):
    """
    Compute terrain-corrected clear-sky solar irradiance (W/m^2)
    using simplified Solis model + topographic corrections.

    Parameters:
    - aod550: Aerosol Optical Thickness at 550 nm
    - tcwv: Total column water vapor in kg/m^2
    - elevation: Elevation above sea level in meters
    - air_temperature: Air temperature in Kelvin (optional, not used in this version)
    - solar_zenith_deg: Solar zenith angle in degrees
    - slope_deg: Terrain slope in degrees
    - aspect_deg: Terrain aspect in degrees (0 = N, 180 = S)
    - solar_azimuth_deg: Solar azimuth angle in degrees
    - svf: Sky view factor (0 to 1), default = 1 (full sky visible)
    - direct_ratio: Fraction of direct radiation in total, default = 0.8

    Returns:
    - GHI (W/m^2): terrain-corrected global horizontal irradiance
    """
    G_sc = 1367.0  # Solar constant (W/m^2)
    deg2rad = np.pi / 180

    # Convert AOD to 700 nm using Angström exponent
    aod700 = aod550 * (700 / 550) ** -1.3
    tcwv_cm = tcwv / 10.0

    theta_z_rad = solar_zenith_deg * deg2rad
    slope_rad = slope_deg * deg2rad
    aspect_rad = aspect_deg * deg2rad
    solar_azimuth_rad = solar_azimuth_deg * deg2rad

    # Simplified transmittance model
    tau_clear = np.exp(-0.9 * aod700) * np.exp(-0.15 * tcwv_cm)
    ghi_flat = G_sc * np.cos(theta_z_rad) * tau_clear

    # Elevation correction
    ghi_elev = ghi_flat * (1 + 0.06 * (elevation / 1000.0))

    # Terrain incidence angle
    cos_incidence = (
        np.cos(theta_z_rad) * np.cos(slope_rad) +
        np.sin(theta_z_rad) * np.sin(slope_rad) * np.cos(solar_azimuth_rad - aspect_rad)
    )
    cos_incidence = np.maximum(cos_incidence, 0)

    # Direct and diffuse components
    direct_flat = ghi_elev * direct_ratio
    diffuse_flat = ghi_elev * (1 - direct_ratio)

    direct_terrain = direct_flat * cos_incidence
    diffuse_terrain = diffuse_flat * svf

    return direct_terrain + diffuse_terrain

In [ ]:
import numpy as np

SOLAR_CONST = 1361  # W/m²

def compute_tilted_irradiance_cloudy(
    aod550, tcwv, elevation,
    solar_zenith_deg, slope_deg, aspect_deg, solar_azimuth_deg,
    era5_ghi=None,
    svf=1.0, albedo=0.2
):
    """
    Compute global tilted irradiance (GTI, W/m²)
    under clear-sky or cloudy conditions.
    
    If ERA5_GHI is provided, applies a cloudiness correction
    as described in literature (ratio method).
    """

    deg2rad = np.pi / 180
    theta_z = solar_zenith_deg * deg2rad
    slope = slope_deg * deg2rad
    aspect = aspect_deg * deg2rad
    phi_s = solar_azimuth_deg * deg2rad

    # --- Clear-sky horizontal irradiance ---
    aod700 = aod550 * (700 / 550) ** -1.3
    tcwv_cm = tcwv / 10.0
    tau_clear = np.exp(-0.9 * aod700) * np.exp(-0.15 * tcwv_cm)
    ghi_flat = SOLAR_CONST * np.cos(theta_z) * tau_clear
    ghi_flat = np.maximum(ghi_flat, 0)

    # Elevation correction
    ghi_elev = ghi_flat * (1 + 0.06 * (elevation / 1000.0))

    # --- Direct/diffuse split (Erbs-like) ---
    kt = ghi_elev / (SOLAR_CONST * np.cos(theta_z) + 1e-6)
    if kt <= 0.22:
        diffuse_fraction = 1 - 0.09 * kt
    elif kt <= 0.80:
        diffuse_fraction = 0.9511 - 0.1604*kt + 4.388*kt**2 - 16.638*kt**3 + 12.336*kt**4
    else:
        diffuse_fraction = 0.165
    diffuse_fraction = np.clip(diffuse_fraction, 0, 1)

    diffuse_flat = ghi_elev * diffuse_fraction
    direct_flat = ghi_elev - diffuse_flat

    # --- Cloud correction ---
    if era5_ghi is not None:
        cloud_factor = era5_ghi / (ghi_elev + 1e-6)
        cloud_factor = np.clip(cloud_factor, 0, 1.2)  # cap overshoots
        diffuse_flat *= cloud_factor
        direct_flat *= cloud_factor
        ghi_elev = diffuse_flat + direct_flat  # recombine

    # --- Terrain incidence correction ---
    cos_incidence = (
        np.cos(theta_z) * np.cos(slope) +
        np.sin(theta_z) * np.sin(slope) * np.cos(phi_s - aspect)
    )
    cos_incidence = np.maximum(cos_incidence, 0)

    direct_tilted = direct_flat * cos_incidence / np.maximum(np.cos(theta_z), 1e-6)
    diffuse_tilted = diffuse_flat * svf * (1 + np.cos(slope)) / 2
    reflected = ghi_elev * albedo * (1 - np.cos(slope)) / 2

    return direct_tilted + diffuse_tilted + reflected
